In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy import MetaData
from configparser import ConfigParser
import os
import spacy
nlp = spacy.load('en')
import re
import phonenumbers
import contextlib

from imwithdata.es_etl.issues_actions import (
    issues,
    actions,
    state_regex,
    city_regex,
    web_url_regex,
    date_include_regex,
    date_exclude_regex,
    leg_name_regex,
    leg_twitter_regex
)

In [2]:
config_file = os.path.join(os.pardir,'config','config.ini')

def get_ini_vals(ini_file, section):
    config = ConfigParser()
    config.read(ini_file)
    return config[section]

mysql_creds = get_ini_vals(config_file, 'mysql')


In [3]:
engine = create_engine("""mysql+pymysql://{user}:{password}@{host}:{port}/{db}""".format(user=mysql_creds['user'],
                                                                                 password=mysql_creds['password'],
                                                                                 host=mysql_creds['host'],
                                                                                 port=mysql_creds['port'],
                                                                                 db=mysql_creds['database']
                                                                                )
                      )
conn = engine.connect()


In [4]:
metadata = sqlalchemy.MetaData()

In [13]:
### DELETE EXISTING TABLES

meta = MetaData()

with contextlib.closing(engine.connect()) as con:
    trans = con.begin()
    for table in reversed(meta.sorted_tables):
        con.execute(table.delete())
    trans.commit()

In [4]:
final_data = pd.read_csv('/Users/brosskatz/PycharmProjects/rzst/w210_imwithdata/imwithdata/data/static_data/final_data_example.csv')

In [5]:
# final_data.head()
final_data.columns

Index(['Unnamed: 0', 'issue', 'action', 'id', 'es_score', 'total_score',
       'tweet', 'tweet_timestamp', 'query_timestamp', 'tweet_user',
       'tweet_cities', 'tweet_states', 'tweet_urls', 'tweet_phone_numbers',
       'tweet_dates_ref', 'tweet_legislator_names',
       'tweet_legislator_handles'],
      dtype='object')

In [6]:
final_data.drop(final_data.columns[0], axis=1,inplace=True)

In [7]:
final_data.columns

Index(['issue', 'action', 'id', 'es_score', 'total_score', 'tweet',
       'tweet_timestamp', 'query_timestamp', 'tweet_user', 'tweet_cities',
       'tweet_states', 'tweet_urls', 'tweet_phone_numbers', 'tweet_dates_ref',
       'tweet_legislator_names', 'tweet_legislator_handles'],
      dtype='object')

In [8]:
final_data['action'].unique()

array(['charity', 'protest', 'petition', 'gathering', 'boycott',
       'advocate', 'vote', 'townhall'], dtype=object)

In [57]:
# event_mask =( ((final_data['action'] == 'protest') | (final_data['action'] == 'gathering' ) | (final_data['action'] == 'townhall')
#               | (final_data['action'] == 'boycott'))
#             )

In [9]:
actions = final_data.sort('total_score',ascending=[0])

/Users/brosskatz/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [10]:
actions.head()

,issue,action,id,es_score,total_score,tweet,tweet_timestamp,query_timestamp,tweet_user,tweet_cities,tweet_states,tweet_urls,tweet_phone_numbers,tweet_dates_ref,tweet_legislator_names,tweet_legislator_handles
584,climate,advocate,AVsaypZnMNAj8foZRgP9,18.827164,25.827164,Join @NRDC &amp; sign petition to defend our e...,2017-03-29T15:58:41.000Z,2017-03-29T21:59:18.900874,KatabreKuika,[],[],['https://t.co/PwbVSJzSyW'],[],[],[],[]
535,womens_right,advocate,AVsbiUtmMNAj8foZRy9y,19.624722,25.624722,Protect Reproductive Health And Rights #sign ...,2017-03-29T19:26:59.000Z,2017-03-29T21:59:18.900874,susan_marie_ll,[],[],['https://t.co/50eRJVFMgX'],[],[],[],[]
688,immigrants,advocate,AVsYHdxXMNAj8foZQ0Cx,26.235420,25.235420,Sign Petition gov of Sanctuary Cities Held Leg...,2017-03-29T03:30:48.000Z,2017-03-29T21:59:18.900874,GuthrieWilson,[],[],['https://t.co/LVlw9L65ZZ'],[],[],[],[]
682,voting,advocate,AVsaSwioMNAj8foZRS-z,16.916447,22.916447,Let's go people sign petition for this Union t...,2017-03-29T13:39:22.000Z,2017-03-29T21:59:18.900874,OnlineTalkShow3,[],[],[],[],[],[],[]
692,immigrants,advocate,AVscHk78MNAj8foZSA1G,16.620247,22.620247,Call Your Senator: Oppose Trump’s Nominee http...,2017-03-29T22:09:44.000Z,2017-03-29T21:59:18.900874,eddyrogerparker,[],[],['https://t.co/k8onyrDE4S'],[],[],[],[]


In [11]:
issue_list = actions['issue'].tolist()
action_list = actions['action'].tolist()
id_list = actions['id'].tolist()

es_score_list = actions['es_score'].tolist()
total_score_list = actions['total_score'].tolist()
tweet_list = actions['tweet'].tolist()
tweet_timestamp_list = actions['tweet_timestamp'].tolist()
query_timestamp_list = actions['query_timestamp'].tolist()
user_list = actions['tweet_user'].tolist()



In [22]:
tweet_list[:10]

['Take Action: Sign Petition to Confirm you Still Want a Secure Border which includes Building a Wall Immediately https://t.co/IGZiJsfDVE',
 'Take Action: Sign Petition to Demand Congress Hire Thousands Border Patrol Agents And Ensure They Are Fully Equipped https://t.co/2k4QMSJmH1',
 'Join @NRDC &amp; sign petition to defend our environment from Trump and his fossil fuel allies in Congress. https://t.co/PwbVSJzSyW',
 'Protect Reproductive Health And Rights  #sign #petition to Stand With Planned Parenthood &gt; https://t.co/50eRJVFMgX',
 'Sign Petition gov of Sanctuary Cities Held Legally and Financially Responsible  for Crimes Committed by Illegals https://t.co/LVlw9L65ZZ',
 "Let's go people sign petition for this Union to put Trump out.Because the rules of the election was broken. Which was voting equality rights",
 'Call Your Senator: Oppose Trump’s Nominee https://t.co/k8onyrDE4S',
 'Sign the #IStandWithPP petition and learn how to call your Member of Congress! Show us what @PPact 

In [13]:
time_regex = re.compile(r'\d{1,2}(?:(?:am|pm)|(?::\d{1,2})(?:am|pm)?)', re.IGNORECASE)

In [18]:
dates = []
start_times = []
end_times = []
cities = []
states = []
legislators = []
legislator_handles = []
phone_numbers = []
titles = []
all_urls = []

for i, tweet in enumerate(tweet_list):
    date = ''
    start_time = ''
    end_time = ''
    city = ''
    state = ''
    legislator = ''
    legislator_handle = ''
    phone_number = ''
    url = ''
    title = ''
    
    doc = nlp(tweet)
    all_dates = [doc.text for doc in doc.ents if doc.label_ == 'DATE']
    date_matches = re.findall(date_include_regex, ' '.join(all_dates))
    ### EXCLUDE SOME DIRTY DATES FROM TWITTER THAT SPACY MISTAKENLY INCLUDES    
    if date_matches:
        date = all_dates[0]
    dates.append(date)
        
    times = re.findall(time_regex,tweet)
    if times:
        start_time = times[0]
    if len(times) > 1:
        end_time = times[1]
    start_times.append(start_time)
    end_times.append(end_time)
    
    tweet_cities = re.findall(city_regex,tweet)
    if tweet_cities:
        tweet_cities = list(set([city.title() for city in tweet_cities]))
        if len(tweet_cities) == 1:
            city = tweet_cities[0]
        else:
            city = '; '.join(tweet_cities)
    cities.append(city)
    
    tweet_states = re.findall(state_regex,tweet)
    if tweet_states:
        tweet_states = list(set(tweet_states))
        if len(tweet_states) == 1:
            state = tweet_states[0]
        else:
            state = '; '.join(tweet_states)
    states.append(state)
    
    tweet_legislators = re.findall(leg_name_regex,tweet)
    if tweet_legislators:
        if len(tweet_legislators) == 1:
            legislator = tweet_legislators[0]
        else:
            legislator = '; '.join(tweet_legislators)
    legislators.append(legislator)
    
    tweet_leg_handles = re.findall(leg_twitter_regex,tweet)
    if tweet_leg_handles:
        if len(tweet_leg_handles) == 1:
            legislator_handle = tweet_leg_handles[0]
        else:
            legislator_handle = '; '.join(tweet_leg_handles)
    legislator_handles.append(legislator_handle)
    
    if phonenumbers.PhoneNumberMatcher(tweet, "US"):
        for i,match in enumerate(phonenumbers.PhoneNumberMatcher(tweet, "US")):
            if i == 0:  
                phone_number = phonenumbers.format_number(match.number,
                                                            phonenumbers.PhoneNumberFormat.NATIONAL)
    phone_numbers.append(phone_number)
    
    urls = ''
    tweet_urls = re.findall(web_url_regex,tweet)
    if tweet_urls:
        tweet_urls = list(set(tweet_urls))
        if len(tweet_urls) == 1:
            urls = tweet_urls[0]
        else:
            urls = '; '.join(tweet_urls)
    all_urls.append(urls)
    
    
    title = tweet
    if tweet_urls:
        for urly in tweet_urls:
            title = title.replace(urly,'')
    titles.append(title)
        
#     print (date, start_time, end_time, city, state, tweet )
    
        
                
#                 print((final_data['action'] == 'protest'),, tweet)

In [21]:
print (len(tweet_list))
print (len(dates))
print (len(start_times))
print (len(end_times))
print (len(cities))
print (len(legislators))
print (len(legislator_handles))
print (len(phone_numbers))
print (len(titles))

906
906
906
906
906
906
906
906
906


In [17]:
print(all_urls[:3000])

['https://t.co/PwbVSJzSyW', 'https://t.co/50eRJVFMgX', 'https://t.co/LVlw9L65ZZ', '', 'https://t.co/k8onyrDE4S', 'https://t.co/MniJtpTVqC', 'https://t.co/HmWRnZgm3A ; https://t.co/fM34H1s0H8', 'https://t.co/CwimrNvlIG', 'https://t.co/0h0wCSjz1P', 'https://t.co/q1WOWOEZx4', 'https://t.co/2zA0tWztuh', 'https://t.co/eUU1oXEVZM ; https://t.co/banu9Oobda', 'https://t.co/qU1hE5pxFx', 'https://t.co/IrleAS2hgD', 'https://t.co/wOn1GNvqKv ; https://t.co/su9W90ks20', 'https://t.co/fD5BIyUctC ; https://t.co/oqN6EK4JrZ', '', 'https://t.co/okYRXnQOMd', 'https://t.co/v8Ea7xe6L2', 'https://t.co/Fwgj1RGzFF', 'https://t.co/LR0D9LdsJ1', 'https://t.co/TWJ9uEF29r', 'https://t.co/2GPNv2zCEI', 'https://t.co/VjetHcE53P', 'https://t.co/vT8pBjbDte', 'https://t.co/Mibeuda8lH', 'https://t.co/OgXZENTpwF', 'https://t.co/9kQIlPSqnG ; https://t.co/NjUcNqjfV1', 'https://t.co/qa4NAflUmI', 'https://t.co/6NNxkU8MDq', 'https://t.co/PfKnDoVvJM', 'https://t.co/RruNs5a2av', 'https://t.co/sY2aDylCOz', 'https://t.co/AYTj7qkYkV

In [23]:
prepped_to_sql = pd.DataFrame(
                        {'id': id_list,
                         'title': [title.encode('utf-8') for title in titles],
                         'description': [tweet.encode('utf-8') for tweet in tweet_list],
                         'action': [action.encode('utf-8') for action in action_list],
                         'issue': [issue.encode('utf-8') for issue in issue_list],
                         'total_score':total_score_list,
                         'relevance_score':es_score_list,
                         'legislators': legislators,
                         'legislator_twitter': legislator_handles,
                         'city':cities,
                         'state':states,
                         'phone_number': phone_numbers,
                         'urls': [url.encode('utf-8') for url in all_urls],
                         'date_of_action': dates,
                         'announced_date': tweet_timestamp_list,
                         'query_date':query_timestamp_list
                        })


# * title
# * description
# * action
# * issue
# * total_score
# * relevance_score
# * legislators
# * city
# * state
# * phone_numbers
# * announced_date
# * query_date


In [25]:
prepped_to_sql.to_sql('rzst_action',conn,if_exists='replace',index=False)

In [26]:
metadata = sqlalchemy.MetaData(conn)

In [27]:
metadata.tables.keys()

dict_keys([])